# Part One: Scrape NASA
Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text. 
Assign the text to variables to reference later.

In [3]:
#Import dependencies
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser
import time

In [6]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [9]:
db = client.mars_db
collection = db.news
mars_news = db.news.find()


In [10]:

url = 'https://mars.nasa.gov/news/'

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'lxml'
soup = bs(response.text, 'lxml')


In [13]:
headlines = soup.find_all('div', class_='content_title')
latest_headline = headlines[0].text
latest_headline = latest_headline.strip()
latest_headline

"NASA Readies Perseverance Mars Rover's Earthly Twin"

In [19]:

article = soup.find_all('div', class_='image_and_description_container')
# paragraph

latest_paragraph = article[0]
latest_paragraph = latest_paragraph.find('div', class_='rollover_description_inner')
latest_paragraph = latest_paragraph.text.strip()
latest_paragraph


"Did you know NASA's next Mars rover has a nearly identical sibling on Earth for testing? Even better, it's about to roll for the first time through a replica Martian landscape."

In [22]:
mars_news_dict = {
    "latest_headline": latest_headline,
    "latest_paragraph": latest_paragraph
}

db.news.insert_one(mars_news_dict)

# Part Two: Scrape JPL for featured image
Visit the url for JPL Featured Space Image here. 
Use splinter to navigate the site and 
Find the image url for the current Featured Mars Image and assign the url string to a variable called featured_image_url.

In [27]:
executable_path = {'executable_path': r'C:\Users\diamo\Bootcamp\chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [28]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)
browser.click_link_by_partial_text('FULL IMAGE')
time.sleep(5)
browser.click_link_by_partial_text('more info')

In [29]:
html = browser.html
soup = bs(html, 'html.parser')
image_url = soup.find('figure', class_='lede').a['href']
image_url

'/spaceimages/images/largesize/PIA16227_hires.jpg'

In [33]:
base_url = 'https://www.jpl.nasa.gov'
full_image_url = base_url + image_url
full_image_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA16227_hires.jpg'

In [35]:
image = {
    "full_image_url": full_image_url
}
#Put into db w/its own collection? 
db.image.insert_one(image)

# Mars Facts
Visit the Mars Facts webpage here and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
Use Pandas to convert the data to a HTML table string.

In [49]:
mars_url = 'https://space-facts.com/mars/'
mars_facts = pd.read_html(mars_url)[0]
# mars_facts
mars_facts_html = mars_facts.to_html()
mars_facts_html

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>0</th>\n      <th>1</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td>\n    <